# 📘 OpenAI API Quick Start

> **주의:** Colab 런타임을 재시작하면 환경 변수가 초기화되므로 `OPENAI_API_KEY`를 다시 설정해야 합니다.


## 환경 준비 및 API 키 설정

In [ ]:
pip install openai

- Colab 메뉴 `보안 비밀(열쇠 아이콘)`에 `OPENAI_API_KEY`를 저장해 두었다면 자동으로 불러옵니다.
- 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

## OpenAI API 시작하기
- 링크: https://platform.openai.com/docs/quickstart
- OpenAI API를 사용하면 챗GPT처럼 프롬프트에 따라 다양한 텍스트(코드, 수식, JSON, 자연어 등)를 간편하게 생성할 수 있습니다.

### 텍스트 생성
- 아래 코드를 실행하면, 프롬프트에 대한 OpenAI API의 응답 결과를 바로 확인할 수 있습니다.

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

### 이미지와 파일 분석

- 이미지 URL, 업로드한 파일, PDF 문서를 모델에 직접 전달하여 텍스트를 추출하고, 콘텐츠를 분류하거나, 시각적 요소를 감지할 수 있습니다.

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "이 이미지에서 어떤 팀들이 경기를 하고 있나요?",
                },
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/3/3b/LeBron_James_Layup_%28Cleveland_vs_Brooklyn_2018%29.jpg"
                }
            ]
        }
    ]
)

print(response.output_text)

### 도구(tool)로 모델 확장
도구를 연결해 모델이 외부 데이터와 함수에 접근하도록 할 수 있습니다. 웹 검색이나 파일 검색 같은 기본 제공 도구를 사용하거나, API 호출·코드 실행·서드파티 시스템 통합을 위한 도구를 직접 정의할 수도 있습니다.

- 웹 검색 예제

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    tools=[{"type": "web_search"}],
    input="오늘 있었던 긍정적인 뉴스는 무엇이었나요?"
)

print(response.output_text)

- 파일 검색 예제

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input="LLM에 대해 설명해줘",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["<vector_store_id>"]
    }]
)
print(response)

- 함수 호출 (function calling) 예제

In [ ]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Seoul, Korea",
                }
            },
            "required": ["location"],
            "additionalProperties": False,
        },
        "strict": True,
    },
]

response = client.responses.create(
    model="gpt-5",
    input=[
        {"role": "user", "content": "What is the weather like in Busan today?"},
    ],
    tools=tools,
)

print(response.output[0].to_json())

### 스트림 응답
서버 전송 스트리밍 이벤트를 사용해 결과가 생성되는 즉시 표시할 수 있습니다.

In [ ]:
from openai import OpenAI
client = OpenAI()

stream = client.responses.create(
    model="gpt-5",
    input=[
        {
            "role": "user",
            "content": "'간장 공장 공장장은 강 공장장'을 열 번 빠르게 말해봐.",
        },
    ],
    stream=True,
)

for event in stream:
    print(event)